In [ ]:
%load_ext autoreload
%autoreload 1
%aimport utils_timeline_viz
from utils_timeline_viz import *
from utils_read_parsing import *
%matplotlib inline
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 7, 4
from tabulate import tabulate

## Loading and preparing survey data

In [ ]:
pre = get_pre_survey()

In [ ]:
pre.head(2)

In [ ]:
#cleaning up columns
old_columns = list(pre.columns)
new_columns = [c.split(']')[0].replace('[','') for c in old_columns]
for i,(c,d) in enumerate(zip(new_columns,old_columns)):
    if new_columns.count(c)>1:
        new_columns[i] = d
print new_columns
pre.columns = new_columns

## Keep records if analyzable ids

In [ ]:
ids = set(get_students_to_analyze_log())

In [ ]:
import collections
ids_pre = list(pre['id'])
print len(ids_pre), len(set(ids_pre))
print [item for item, count in collections.Counter(ids_pre).items() if count > 1]
print "We have duplicates, one from the collision and one from... who knows!"
print 17931169 in ids

In [ ]:
print len(ids&set(ids_pre)),"ids in pre and analyzable list"
print  len(ids - set(ids_pre)),"ids not in pre"
print len(set(ids_pre)-ids),"ids not analyzable ids (not worrisome)"

In [ ]:
ids_to_sort = set(ids - set(ids_pre))
for idd in ids_to_sort:
    print idd, int(str(idd)[1:]) in ids_pre
print len(ids_to_sort)

In [ ]:
def ids_logs_to_pre(studentid):
    if studentid in [15055169,15444164,19780165,17868168,16901162,10561164,13981167,10192168,15298166,11394167,14055131]:
        return studentid-10000000
    elif studentid == 12406161:
        return 12406816
    elif studentid == 18162165:
        return 18162136
    else:
        return studentid
ids_match_pre = set([ids_logs_to_pre(i) for i in ids])
print len(ids_match_pre)
print len(ids_match_pre&set(ids_pre))

In [ ]:
#using internal id to remove the student with two pres,one in a good session with logs, one not
#id = 17931169
pre_analyzeable = pre[(pre['id'].isin(ids_match_pre))&(pre['Internal ID']!=86221654)]
pre_analyzeable.to_csv('pre_survey_results.txt', sep='\t')
len(pre_analyzeable)

## Let's analyze!

In [ ]:
ax = pre_analyzeable['prior_sims_phet'].value_counts().plot(kind='bar')
for p in ax.patches:
    ax.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005), fontsize=25)

In [ ]:
ax = pre_analyzeable['prior_number_virtual_labs'].value_counts().plot(kind='bar')
for p in ax.patches:
    ax.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005), fontsize=25)

In [ ]:
ax = pre_analyzeable['prio_sims_other'].value_counts().plot(kind='bar')
for p in ax.patches:
    ax.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005), fontsize=25)

In [ ]:
lab_choices = ['[High school level laboratory]','[First year undergraduate physics laboratory]','[First year undergraduate chemistry laboratory]','[Higher level physics labs]','[Higher level chemistry labs]']
p = []
p.append(lab_choices)
p.append([len(pre_analyzeable[pre_analyzeable['[prior_lab] What lab courses are you presently taking or have taken in the past? Check all that apply. '+choice]==1]) for choice in lab_choices])
print tabulate(p)
# len(pre_analyzeable[pre_analyzeable['[prior_lab] What lab courses are you presently taking or have taken in the past? Check all that apply. [High school level laboratory]']==1])

In [ ]:
pre_analyzeable.columns

In [ ]:
#Other lab courses taken by students
print  [v for v in pre_analyzeable['prior_lab'].values if str(v) != 'nan']